# <font color='blue'> Validação cruzada e aleatoridade inicial </font>

## <font color='blue'> A influência da aleatoriedade na validação do modelo </font>

### <font color='blue'> Baixando os dados: </font>

In [157]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


### <font color='blue'> Separrando os modelos de treino e teste: </font>

In [158]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

Treinaremos com 7500 elementos e testaremos com 2500 elementos


### <font color='blue'> Decidindo oq é uma taxa de acerto aceitável: </font>

In [159]:
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print("A acurácia do dummy stratified foi de %.2f%%" % acuracia)

A acurácia do dummy stratified foi de 58.00%


### <font color='blue'> Rodando uma arvore de decisão: </font>

In [160]:

from sklearn.tree import DecisionTreeClassifier

SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print ("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 71.92%


In [161]:
x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

SEED = 5
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia do dummy stratified foi %.2f%%" % acuracia)

Treinaremos com 7500 elementos e testaremos com 2500 elementos
A acurácia do dummy stratified foi 76.84%


## <font color='blue'> Usando e avaliando com o cross validate: </font>

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html

In [162]:
from sklearn.model_selection import cross_validate

SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv=3)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation, 3 = [%.2f , %.2f]" % ((media - 2 * desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

Accuracy com cross validation, 3 = [74.99 , 76.57]


In [163]:
SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv=10)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation, 3 = [%.2f , %.2f]" % ((media - 2 * desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

Accuracy com cross validation, 3 = [74.24 , 77.32]


In [164]:
SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv=5) # Usar 5 já é o suficiente.
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation, 3 = [%.2f , %.2f]" % ((media - 2 * desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

Accuracy com cross validation, 3 = [75.21 , 76.35]


# <font color='blue'> KFold e aleatoridade: </font>

## <font color='blue'> KFold com aleatorização: </font>

#### <font color='black'> Aleatoriedade no cross validate: </font>

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html?highlight=kfold#sklearn.model_selection.KFold

In [165]:
def imprime_resultados(results):
    media = results['test_score'].mean()
    desvio_padrao = results['test_score'].std()
    print("Accuracy médio: %.2f" % (media * 100))
    print("Accuracy intervalo: [%.2f , %.2f]" % ((media - 2 * desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

In [166]:
from sklearn.model_selection import KFold

SEED = 158020
np.random.seed(SEED)

cv = KFold(n_splits= 10)
modelo = DecisionTreeClassifier(max_depth= 2)
results = cross_validate(modelo, x, y, cv= cv) # Usar 5 já é o suficiente.
imprime_resultados(results)

Accuracy médio: 75.78
Accuracy intervalo: [74.37 , 77.19]


In [167]:
SEED = 158020
np.random.seed(SEED)

cv = KFold(n_splits= 10, shuffle= True)
modelo = DecisionTreeClassifier(max_depth= 2)
results = cross_validate(modelo, x, y, cv= cv) # Usar 5 já é o suficiente.
imprime_resultados(results)

Accuracy médio: 75.78
Accuracy intervalo: [73.58 , 77.98]


# <font color='blue'> Estratificação: </font>

## <font color='blue'> Estratificação com validação cruzada: </font>

In [168]:
### Simular situaçâo horrível de azar
### <p> Pode ser "azar" como pode ser uma proporção de exemplos desbalanceado entre as classes</p>

In [169]:
dados_azar = dados.sort_values("vendido", ascending=True)
x_azar = dados_azar[["preco", "idade_do_modelo","km_por_ano"]]
y_azar = dados_azar["vendido"]
dados_azar

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704
...,...,...,...,...
5491,71910.43,1,9,25778.40812
1873,30456.53,1,6,15468.97608
1874,69342.41,1,11,16909.33538
5499,70520.39,1,16,19622.68262


In [170]:
from sklearn.model_selection import KFold

SEED = 158020
np.random.seed(SEED)

cv = KFold(n_splits= 10)
modelo = DecisionTreeClassifier(max_depth= 2)
results = cross_validate(modelo, x_azar, y_azar, cv= cv) # Usar 5 já é o suficiente.
imprime_resultados(results)

Accuracy médio: 57.84
Accuracy intervalo: [34.29 , 81.39]


In [171]:
SEED = 158020
np.random.seed(SEED)

cv = KFold(n_splits= 10, shuffle= True)
modelo = DecisionTreeClassifier(max_depth= 2)
results = cross_validate(modelo, x_azar, y_azar, cv= cv) # Usar 5 já é o suficiente.
imprime_resultados(results)

Accuracy médio: 75.76
Accuracy intervalo: [73.02 , 78.50]


https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html?highlight=kfold#sklearn.model_selection.StratifiedKFold

In [172]:
from sklearn.model_selection import StratifiedKFold

SEED = 158020
np.random.seed(SEED)

cv = StratifiedKFold(n_splits= 10, shuffle= True) # Quando tem um desbalanço entre as duas classes é melhor usar o StratifideKFold
modelo = DecisionTreeClassifier(max_depth= 2)
results = cross_validate(modelo, x_azar, y_azar, cv= cv) # Usar 5 já é o suficiente.
imprime_resultados(results)

Accuracy médio: 75.78
Accuracy intervalo: [74.10 , 77.46]


## <font color='blue'> Documentação e boas práticas: </font>

https://scikit-learn.org/stable/modules/classes.html?highlight=model%20selection#module-sklearn.model_selection

In [173]:
# <font color='blue'> Dados agrupáveis: </font>

In [174]:
## <font color='blue'> Gerando dados aleatórios: </font>

In [175]:
### Gerando dados aleatórios de modelo de carro para simulação de agrupamento ao usar nosso estimador

In [176]:
np.random.seed(SEED)
dados['modelo']= dados.idade_do_modelo + np.random.randint(-2,3, size=10000)
dados.modelo = dados.modelo + abs(dados.modelo.min()) + 1 # Tirando o valor negativo e somando 1
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,19
1,40557.96,1,20,12622.05362,22
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,4
4,117384.68,1,4,12770.11290,4


In [177]:
dados.modelo.unique()

array([19, 22, 14,  4, 12, 11, 20,  6, 18,  8,  7, 23, 17, 15, 13, 16, 10,
        5, 21,  9, 24,  2,  3,  1], dtype=int64)

In [178]:
dados.modelo.value_counts()

20    873
19    830
18    788
21    707
17    687
15    669
16    616
22    597
14    577
13    576
12    497
11    397
10    376
23    347
9     320
8     288
7     217
24    199
6     186
5     120
4      66
3      44
2      20
1       3
Name: modelo, dtype: int64

In [179]:
## <font color='blue'> Validação cruzada usando grupos: </font>

In [180]:
#### Rodando nosso classificador:

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupKFold.html

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html

In [181]:
#### Testando validção cruzada com groupKFold

In [182]:
from sklearn.model_selection import GroupKFold

SEED = 158020
np.random.seed(SEED)

cv = GroupKFold(n_splits= 10) # Quando tem um desbalanço entre as duas classes é melhor usar o StratifideKFold
modelo = DecisionTreeClassifier(max_depth= 2)
results = cross_validate(modelo, x_azar, y_azar, cv= cv, groups= dados.modelo) # Usar 5 já é o suficiente.
imprime_resultados(results)

Accuracy médio: 75.76
Accuracy intervalo: [72.91 , 78.60]


In [183]:
# <font color='blue'> Pipeline de treino e validaçaõ: </font>

In [184]:
## <font color='blue'> A importância do pipeline no crossvalidate: </font>

In [185]:
### Cross validation com StandardScaler

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

In [186]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

SEED = 301
np.random.seed(SEED)
# Escalando os X´s

scaler = StandardScaler()
scaler.fit(treino_x)
treino_x_escalado = scaler.transform(treino_x)
teste_x_escalado = scaler.transform(teste_x)
# Rodando o modelo:
modelo = SVC()
modelo.fit(treino_x_escalado, treino_y)
previsoes = modelo.predict(teste_x_escalado)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A Acirácia foi de %.2f%%" % acuracia)

A Acirácia foi de 77.48%


from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits= 10) # Quando tem um desbalanço entre as duas classes é melhor usar o StratifideKFold
modelo = SVC()
results = cross_validate(modelo, x_azar, y_azar, cv= cv, groups= dados.modelo) # Usar 5 já é o suficiente.
imprime_resultados(results)

https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

In [193]:
from sklearn.pipeline import Pipeline

SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
modelo = SVC()

# Com o pipeline pode ser feito um pré-processamento mais do que uma vez.
pipeline = Pipeline([('transformacao',scaler), ('estimador', modelo)])

cv = GroupKFold(n_splits= 10) # Quando tem um desbalanço entre as duas classes é melhor usar o StratifideKFold
results = cross_validate(pipeline, x_azar, y_azar, cv= cv, groups= dados.modelo) # Usar 5 já é o suficiente.
imprime_resultados(results)

Accuracy médio: 76.66
Accuracy intervalo: [73.65 , 79.66]
